In [2]:
import zipfile
import pandas as pd 
import numpy as np
from IPython.display import display
from pprint import pprint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

Tokenization Stage

In [3]:
dataset = pd.read_csv('/content/Output/tweet_emotions.csv')

content = dataset['content']

contentArr = content.to_numpy()


contentArr_length = len(contentArr)

sentiment = dataset['sentiment']
sentimentArr = sentiment.to_numpy()

print(sentimentArr)

sentimentArr_length = len(sentimentArr)

sentimentUniqueArr = sentiment.unique()
print(sentimentUniqueArr)

sentimentArr_unique_length = len(sentimentUniqueArr)

print(sentimentArr_unique_length)

tokenizer = Tokenizer(num_words=1000, oov_token='<oov>')
sentimentTokenizer = Tokenizer(num_words=13)


def vocab_count(arr):
  vocab_size = 0

  for sentences in arr:
    for j in sentences:
      vocab_size += 1

  return vocab_size

  print(vocab_count(contentArr))


def prepare_sentence(sentence):
  return re.sub(r'([{}])'.format(string.punctuation),r' ', sentence)

def tokenization(arr):
  prepared_arr = []
  for sentence in arr:
    newSentence = prepare_sentence(sentence)
    prepared_arr.append(newSentence)
  
  tokenizer.fit_on_texts(prepared_arr)
  sequences = tokenizer.texts_to_sequences(prepared_arr)
  return pad_sequences(sequences)

# Tried to_categorical

def sentimentTokenization(arr):
  prepared_arr = []
  for sentence in arr:
    newSentence = prepare_sentence(sentence)
    prepared_arr.append(newSentence)
  
  sentimentTokenizer.fit_on_texts(prepared_arr)
  sequences = sentimentTokenizer.texts_to_sequences(prepared_arr)
  return pad_sequences(sequences)

content_length = int(0.7 * len(contentArr))
sentiment_length = int(0.7 * len(sentimentArr))


contentNewArr = tokenization(contentArr)
sentimentInNumber = {}
sentimentNewArr = sentimentTokenization(sentimentArr)
# sentimentNewCArr = tf.keras.utils.to_categorical(sentimentNewArr)

# unwanted_idx = []

# for i, sentiment in enumerate(sentimentNewArr):
#   if(sentiment > 13):
#     unwanted_idx.append(sentiment)
# print(len(unwanted_idx))

training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(contentNewArr, sentimentNewArr, test_size=0.3)

training_labels = tf.keras.utils.to_categorical(training_labels)
testing_labels = tf.keras.utils.to_categorical(testing_labels)

print(training_labels)
print(testing_labels)

print(len(testing_labels))


# contentArr = content.to_numpy()

vocab_size = tokenizer.num_words
print(vocab_size)



['empty' 'sadness' 'sadness' ... 'love' 'happiness' 'love']
['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']
13
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
12000
1000


Models

In [ ]:
embedding_dim = 64

model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model1.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))))
model1.add(tf.keras.layers.Dropout(0.3))
model1.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model1.add(tf.keras.layers.Dropout(0.3))
model1.add(tf.keras.layers.Dense(13, activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

history1 = model1.fit(training_sentences, training_labels, batch_size=128, validation_data=(testing_sentences, testing_labels), epochs=20)

model1.save('m1.h5')  
model1.load_weights('m1.h5')


model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model2.add(tf.keras.layers.LSTM(64, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.3))
model2.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model2.add(tf.keras.layers.Dropout(0.3))
model2.add(tf.keras.layers.Dense(13, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

history2 = model2.fit(training_sentences, training_labels, batch_size=128, validation_data=(testing_sentences, testing_labels), epochs=20)

model2.save('m2.h5')  
model2.load_weights('m2.h5')


model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, recurrent_dropout = 0.4, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4), activity_regularizer=tf.keras.regularizers.l2(1e-5))))
model3.add(tf.keras.layers.Dropout(0.3))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, recurrent_dropout = 0.4, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4), activity_regularizer=tf.keras.regularizers.l2(1e-5))))
model3.add(tf.keras.layers.Dropout(0.3))
model3.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model3.add(tf.keras.layers.Dropout(0.3))
model3.add(tf.keras.layers.Dense(13, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

history3 = model3.fit(training_sentences, training_labels, batch_size=128, validation_data=(testing_sentences, testing_labels), epochs=20)

model3.save('m3.h5')  
model3.load_weights('m3.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          64000     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 13)                845       
                                                        

Loading all Models



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

num_models = 3

def get_all_models(num_models):
  all_models = list()
  for i in range(num_models):
    modelFile = '/content/' + 'm' + str(i+1) + '.h5'
    model = keras.models.load_model(modelFile)
    all_models.append(model)
  
  return all_models


models = get_all_models(num_models)


def stacked_predictions(members, x_input):
  stacked_list = list()
  stacked_arr = np.array(stacked_list)

  print(len(stacked_arr))

  for model in members:
    stacked_arr = None
    model_pred = model.predict(x_input)

    if(stacked_arr is None):
      stacked_arr = model_pred
    else:
      stacked_arr = np.concatenate((stacked_arr, model_pred), axis=0)

  stacked_arr = stacked_arr.reshape((stacked_arr.shape[0], stacked_arr.shape[1] * stacked_arr.shape[2]))

  return stacked_arr

stacked_prediction = stacked_predictions(models, testing_sentences)

def fit_stacked_model(members, x_test, y_test):
  model = LogisticRegression()
  model.fit(stacked_prediction, y_test)

  return model

final_model = fit_stacked_model(models, testing_sentences, testing_labels)

def final_predictions(members, x_test, y_test):
  final_predictions = final_model.predict(stacked_prediction)
  return final_predictions

y_test_predictions = final_predictions(models, testing_sentences, testing_labels)

print(y_test_predictions)
model_accuracy_score = accuracy_score(testing_labels, y_test_predictions)

print(model_accuracy_score)




  




    

0


IndexError: ignored